## Imports

In [1]:
import os
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
from torch import optim
from torchvision import models,transforms,datasets
import pandas as pd
from torch.autograd import Variable
import bcolz

In [2]:
# use_gpu = False
use_gpu = torch.cuda.is_available()
def gpu(x,use_gpu=use_gpu):
    if use_gpu:
        return x.cuda()
    else:
        return x

## Data processing

### Loading dataset

In [3]:
data_dir = './Dataset'

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

vgg_format = transforms.Compose([
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                normalize,
            ])

class ImageFolderWithPaths(datasets.ImageFolder):
    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

dsets = {x: ImageFolderWithPaths(os.path.join(data_dir, x), vgg_format)
         for x in ['train', 'val', 'test']}
    
dset_sizes = {x: len(dsets[x]) for x in ['train', 'val','test']}
dset_classes = dsets['train'].classes
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=64,
                                               shuffle=True, num_workers=6)
                for x in ['train', 'val', 'test']}

### Clean text data

In [4]:
# clean IngredientList.txt and output the doublets
ing_list = './IngredientList.txt'
text_file = open("./IngredientList.txt", "r")
lines = text_file.read().split('\n')
ing_dict = {}
i = 0
for l in lines:
    ing_dict[i] = l
    i += 1
for key, val in ing_dict.items():
    val = val.lower()
    if "\ufeff" in val:
        val = val.replace('\ufeff','')
    if "of" in val:
        val = val.replace('of ','')
    if "chiffonade" in val:
        val = val.replace('chiffonade','sliced')
    if "slices" in val:
        val = val.replace('slices','sliced')
    if 'julienned' in val:
        val = val.replace('julienned', 'shredded')
    val = val.replace(' ', '')
    ing_dict[key] = val

cut_set = {0: "batonnet", 1: "minced", 2: "sliced", 3: "crushed", 4: "shredded", 5: "brunoisediced", 6: "chopped", 7:"chunks", 8: "fried", 9: "steamed", 10: "boiled", 11: "pickled", 12: "seared", 13: "dried"}

new_ing_dict = {}
ing_set = {}
i, j = 0, 0

for idx, val in ing_dict.items():
    new_ing_dict[idx] = [-1,-1]    #note -1 if missing data
    # find cut
    for jdx, cut in cut_set.items():
        if cut in val:
            new_ing_dict[idx][0] = jdx
            val = val.replace(cut, '')
    # find ingre
    if not val in ing_set.values():
        ing_set[j] = val
        j += 1
        new_ing_dict[idx][1] = j
    else:
        new_ing_dict[idx][1] = list(ing_set.keys())[list(ing_set.values()).index(val)]

In [5]:
ing_set

{0: 'greenonion',
 1: 'whiteonion',
 2: 'ham',
 3: 'carrot',
 4: 'hobblockscarrot',
 5: 'blacksesame',
 6: 'whitesesame',
 7: 'pepper',
 8: 'wholegreenpepper',
 9: 'chineseparsleycoriander',
 10: 'beansprouts',
 11: 'bacon',
 12: 'tomatosclices',
 13: 'cherrytomato',
 14: 'greenvegetables',
 15: 'cornkernels',
 16: 'cornblocks',
 17: 'bittergourd',
 18: 'broccoli',
 19: 'cauliflower',
 20: 'toast',
 21: 'bread',
 22: 'chinesemahogany',
 23: 'crabsticks',
 24: 'bambooshootstips',
 25: 'chivepieces',
 26: 'chives',
 27: 'onion',
 28: 'pumpkinblocks',
 29: 'groundnutkernels',
 30: 'lemon',
 31: 'shelledfreshshrimps',
 32: 'seashrimp',
 33: 'freshshrimp',
 34: 'crayfish',
 35: 'cucumber',
 36: 'hobblockscucumber',
 37: 'hotanddrychili',
 38: 'hotanddrypepper',
 39: 'hotanddrypepperpowder',
 40: 'purplecabbage',
 41: 'chinesecabbage',
 42: 'blackfungus',
 43: 'perillacrispatanaka',
 44: 'ginger',
 45: 'scrambledegg',
 46: 'eggcake',
 47: 'eggcrepe',
 48: 'spicedcornedegg',
 49: 'eggyolk',
 

In [6]:
new_ing_dict

{0: [1, 1],
 1: [2, 0],
 2: [12, 0],
 3: [-1, 2],
 4: [5, 3],
 5: [4, 2],
 6: [-1, 2],
 7: [2, 2],
 8: [5, 4],
 9: [2, 3],
 10: [0, 3],
 11: [4, 3],
 12: [-1, 5],
 13: [-1, 6],
 14: [-1, 7],
 15: [3, 8],
 16: [4, 7],
 17: [2, 7],
 18: [12, 7],
 19: [-1, 7],
 20: [-1, 9],
 21: [-1, 10],
 22: [-1, 11],
 23: [-1, 12],
 24: [-1, 13],
 25: [2, 14],
 26: [-1, 13],
 27: [-1, 15],
 28: [2, 14],
 29: [-1, 16],
 30: [-1, 17],
 31: [2, 18],
 32: [-1, 19],
 33: [-1, 20],
 34: [-1, 21],
 35: [-1, 22],
 36: [-1, 23],
 37: [-1, 24],
 38: [-1, 25],
 39: [-1, 26],
 40: [6, 27],
 41: [-1, 26],
 42: [4, 28],
 43: [2, 27],
 44: [5, 27],
 45: [-1, 29],
 46: [-1, 30],
 47: [3, 29],
 48: [-1, 31],
 49: [-1, 32],
 50: [13, 33],
 51: [-1, 34],
 52: [-1, 35],
 53: [4, 36],
 54: [0, 35],
 55: [-1, 37],
 56: [2, 35],
 57: [5, 35],
 58: [3, 38],
 59: [-1, 39],
 60: [-1, 40],
 61: [2, 41],
 62: [-1, 42],
 63: [-1, 43],
 64: [-1, 44],
 65: [2, 45],
 66: [4, 44],
 67: [6, 44],
 68: [-1, 46],
 69: [-1, 47],
 70: [3, 4

## Model building

In [7]:
def norm(input, p=2, dim=1, eps=1e-12):
    return input / input.norm(p,dim,keepdim=True).clamp(min=eps).expand_as(input)

### Embedding

In [8]:
class embedding(nn.Module):
    def __init__(self):
        super(embedding, self).__init__()
        # new feature
        self.feature_emb = nn.Sequential(
            nn.Linear(512, 172), 
            nn.Tanh(),
        )
        # ingredient
        self.ingredient_emb = nn.Sequential(
            nn.Linear(172, 353),
            nn.Softmax(dim=-1),
        )
         # cutting
        self.cutting_emb = nn.Sequential(
            nn.Linear(172, 14),
            nn.Softmax(dim=-1),
        )
    
    def forward(self, x):
        x = x.view(x.size(0), 512, -1)
        x = torch.transpose(x,1,2)
        fea_embedding = self.feature_emb(x)
        fea_embedding = norm(fea_embedding)
        ing_embedding = self.ingredient_emb(fea_embedding)
        cut_embedding = self.cutting_emb(fea_embedding)
        return fea_embedding, ing_embedding, cut_embedding     # 3-d [batch_size,self_size,49]

### Pooling

In [9]:
def idx_unflatten(tsor, dim):
    b = tsor.cpu().numpy()
    c = []
    for val in b:
        c.append([(int)(val/dim),val%dim])
    c = np.asarray(c)
    c = c.reshape(b.shape)
    new_tensor = torch.from_numpy(c)
    return new_tensor

def pooling(ing_emb, cut_emb):               #input_size = [64,49,353/14]
    # get the i-th region of ingredient j
    ing_max, idx = torch.max(ing_emb, dim=1)    #[64,354]   idx_val in [0,48]

    p_ingre = ing_max
    
    # pooling cutting and cooking method of i-th region
    p_cut = torch.zeros(cut_emb.size(0),ing_emb.size(2),cut_emb.size(2))

    i = 0
    for ing_batch in idx:           #ing_batch    positions of most popular grid of each ingre in a batch, size=354
        for j in range(ing_max.size(1)):
            p = ing_batch[j]
#             for k in range(cut_emb.size(2)):
            p_cut[i,j,:] = cut_emb[i,p,:]
#         print(i)
        i += 1
    return p_ingre, p_cut           # p_ingre:[64,353],  p_cut:[64,353,14]

### Recipe retrival from text

In [10]:
# original ingredient label is represented as a list
# output is a list of doublet(ing, cut)
def recipe_retrival(ingre_label):
    res = []
    for idx, val in enumerate(ingre_label):
        if val == 1:
            doublet = new_ing_dict[idx]             
            res.append(doublet)
    return res

IngreLabel = open('IngreLabel.txt', 'r').read().split('\n')[:-1]   # list of str
for i in range(len(IngreLabel)):
    IngreLabel[i] = IngreLabel[i].split()                    # list of list, element is str

# Get t_ingre from path
def Get_ingre(path):
    ID = path[11]
    if ID == 'r':    #train
        path = path[15:]
    elif ID == 'a':  #val
        path = path[13:]
    else:            #test
        path = path[14:]
    # path has the same format of that in IngreLabel.txt
    for pil in IngreLabel:
        if pil[0] == path:
            ingre = [int(i) for i in pil[1:]]
            ingre = np.asarray(ingre)
            ingre = torch.from_numpy(ingre)
            return ingre

# Get t_cut from t_ingre
def Get_cut(t_ingre):
    t_cut = torch.zeros([cutDim, ingDim])
    recipe = recipe_retrival(t_ingre)
    a,b = zip(*recipe)
    c = list(a)   # cut
    t = list(b)   # ingre
    for i in range(len(c)):
        if c[i] != 0:
            t_cut[c[i],t[i]] = 1 
    t_cut = torch.transpose(t_cut,0,1)
    return t_cut

In [11]:
len(IngreLabel[-1])

354

### All recipes

In [12]:
# Get all recipes from IngreLabel.txt
All_recipe = []
for p in IngreLabel:
    ingre = [int(i) for i in p[1:]]
    r = recipe_retrival(ingre)
    All_recipe.append(r)

In [13]:
All_recipe

[[[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [5, 3], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [5, 3], [-1, 47]],
 [[-1, 47]],
 [[-1, 47], [-1, 207]],
 [[-1, 47]],
 [[1, 1], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[-1, 47]],
 [[1, 1], [-1, 47]],
 [[5, 4], [-1, 6], [-1, 47]],
 [[-1, 13], [-1, 47], [-1, 215]],
 [[-1, 47]],
 [[1, 1], [5, 3], [-1, 47]],
 [[1, 1], [-1, 47]],
 [[5, 3], [-1, 13], [2, 35], [-1, 47], [-1, 215]],
 [[1, 1], [-1, 47]],
 [[3, 8], [-1, 47]],
 [[-1, 47], [-1, 214]],
 [[6, 27], [-1, 47]],
 [[-1, 47]],
 [[1, 1], [-1, 47]],
 [[-1, 10], [-1, 47]

In [14]:
# Get dish name for all recipes
Dish_name = []
for p in IngreLabel:
    tmp = p[0]
    tmp = tmp.split('/')
    Dish_name.append(int(tmp[1]))

In [15]:
min(Dish_name), max(Dish_name)

(1, 172)

### Loss function

In [16]:
#calculate cross entropy between image representation p and target representation t

def loss_function(p_ingre, p_cut, t_ingre, t_cut):
    
    # loss of ingredients
    loss = gpu(nn.BCEWithLogitsLoss())
    t_ingre = gpu(t_ingre.float())
    p_ingre = gpu(p_ingre.float())
    L1 = loss(p_ingre,t_ingre)
    
    # ground-truth ingredient set
    a = t_ingre.nonzero().view(-1)
  
    # conditional cutting tensor
    new_p_cut = torch.zeros(p_cut.size())
    for j in a.cpu().numpy():
        for i in range(p_cut.size(1)):
            new_p_cut[j][i] = p_cut[j][i]
    # loss of cutting
    t_cut = gpu(t_cut.float())
    new_p_cut = gpu(new_p_cut.float())
    L2 = loss(new_p_cut, t_cut)
  
    L = L1 + L2
    return L

### Similarity and prediction

In [17]:
def similarity(Q_ingre, Q_cut, R):
    simi = 0
    for doublet in R:
        ing, cut = doublet[0], doublet[1]
    simi += Q_ingre[ing] + cc_rate * Q_cut[cut][ing]
    simi /= len(R)
  
    return simi

def pred_dish(Q_ingre,Q_cut,all_recipe):
    simi = 0
    res = -1
    for idx, recipe in enumerate(all_recipe):
        tmp = similarity(Q_ingre, Q_cut, recipe)
        if tmp > simi:
            simi = tmp
            res = idx
    return(Dish_name[res])

## Training

### Preconvoluted features

In [18]:
model_vgg = models.vgg16(pretrained=True)
model = gpu(model_vgg.features)

In [19]:
def preconvfeat_t(dataset):
    conv_features = []
    labels_list = []
    paths_list = []
    for data in dataset:
        inputs,labels,paths = data
        inputs = gpu(inputs)
        labels = gpu(labels)
        paths = list(paths)
        x = model(inputs)
        conv_features.extend(x.data.cpu().numpy())
        labels_list.extend(labels.data.cpu().numpy())
        paths_list.extend(paths)
    conv_features = np.concatenate([[feat] for feat in conv_features])
    return (conv_features,labels_list,paths_list)

In [25]:
%%time
conv_feat_train,labels_train, paths_train = preconvfeat_t(dset_loaders['train'])

CPU times: user 11min 24s, sys: 6min 34s, total: 17min 58s
Wall time: 18min


In [26]:
%%time
conv_feat_val,labels_val, paths_val = preconvfeat_t(dset_loaders['val'])

CPU times: user 1min 54s, sys: 1min 5s, total: 2min 59s
Wall time: 3min 1s


In [32]:
%%time
conv_feat_test,labels_test, paths_test = preconvfeat_t(dset_loaders['test'])

CPU times: user 5min 42s, sys: 3min 16s, total: 8min 58s
Wall time: 8min 59s


### Save preconvoluted features

In [20]:
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
def load_array(fname):
    return bcolz.open(fname)[:]

In [29]:
%mkdir ./cross_model

In [33]:
save_array(os.path.join('./cross_model','feat_train.bc'),conv_feat_train)
save_array(os.path.join('./cross_model','labels_train.bc'),labels_train)
save_array(os.path.join('./cross_model','paths_train.bc'),paths_train)
save_array(os.path.join('./cross_model','feat_val.bc'),conv_feat_val)
save_array(os.path.join('./cross_model','labels_val.bc'),labels_val)
save_array(os.path.join('./cross_model','paths_val.bc'),paths_val)
# save_array(os.path.join('./cross_model','feat_test.bc'),conv_feat_test)
# save_array(os.path.join('./cross_model','labels_test.bc'),labels_test)
# save_array(os.path.join('./cross_model','paths_test.bc'),paths_test)

### Loading preconvoluted features

In [21]:
conv_feat_tr = load_array('./cross_model/feat_train.bc')
labels_tr = load_array('./cross_model/labels_train.bc')
paths_tr = load_array('./cross_model/paths_train.bc')
conv_feat_v = load_array('./cross_model/feat_val.bc')
labels_v = load_array('./cross_model/labels_val.bc')
paths_v = load_array('./cross_model/paths_val.bc')
# conv_feat_te = load_array(data_dir+'/cross_model/conv_feat_test.bc')
# labels_te = load_array(data_dir+'/cross_model/labels_test.bc')
# paths_te = load_array(data_dir+'/cross_model/paths_test.bc')

### Data_gen

In [22]:
def data_gen(conv_feat,labels,paths,batch_size=64,shuffle=True):
    labels = np.array(labels)
    newpaths = []
    if shuffle:
        index = np.random.permutation(len(conv_feat))
        conv_feat = conv_feat[index]
        labels = labels[index]
#         k = 0
        for i in index:
            newpaths.append(paths[i])
#             k += 1
    for idx in range(0,len(conv_feat),batch_size):
        yield(conv_feat[idx:idx+batch_size],labels[idx:idx+batch_size],newpaths[idx:idx+batch_size])

### Training embedding layers

In [34]:
batch_size = 64
learning_rate = 1e-2
epochs = 10

optimizer = optim.SGD(embedding().parameters(), learning_rate)

def train_model_emb(model,size_train,size_val,conv_feat_train=None,labels_train=None, paths_train=None, 
                conv_feat_val=None,labels_val=None,paths_val=None,batch_size=64,epochs=1,optimizer=None,
                    train=True,shuffle=True):
    if train:
        model.train()
    else:
        model.eval()

    LOSS = []
    ACC = []
    LOSS_V = []
    ACC_V = []
    
    for epoch in range(epochs):
        batches = data_gen(conv_feat=conv_feat_train,labels=labels_train,paths=paths_train,batch_size=batch_size,
                           shuffle=shuffle)

        total = 0
        running_loss = 0.0
        running_corrects = 0
        
        k = 1
        for inputs,classes,paths in batches:
            inputs, classes = gpu(torch.from_numpy(inputs)), gpu(torch.from_numpy(classes))          
            
            fea_embeddings, ing_embeddings, cut_embeddings  = model(inputs)
            
            loss = 0
            preds = []
            p_ingres,p_cuts = pooling(ing_embeddings, cut_embeddings)
            for i in range(fea_embeddings.size(0)):  #batch_size
                fea_emb = fea_embeddings[i]
                ing_emb = ing_embeddings[i]
                cut_emb = cut_embeddings[i]
                t_ingre = Get_ingre(paths[i])
                t_cut = Get_cut(t_ingre)
                p_ingre = p_ingres[i]
                p_cut = p_cuts[i]
                # loss
                l = loss_function(p_ingre, p_cut, t_ingre, t_cut)
                # similarity, retrival
                pred = pred_dish(p_ingre,p_cut,All_recipe)
#                 loss.append(l)
                loss += l
                preds.append(pred)
                print('{}-th over 64 of batch {}'.format(i+1,k))
#                 print(type(loss))
#                 print(loss)
            preds = gpu(torch.from_numpy(np.asarray(preds)))
#             loss = gpu(torch.autograd.Variable(torch.from_numpy(np.asarray([loss]))))
            
            if optimizer is None:
                raise ValueError('Pass optimizer for train mode')
            optimizer = optimizer
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == classes.data)
            k += 1
        epoch_loss = running_loss / size_train
        epoch_acc = running_corrects.data.item() / size_train
        LOSS.append(epoch_loss)
        ACC.append(epoch_acc)
        print('Epoch: {} Training Loss: {:.4f} Training Acc: {:.4f}'.format(
                     epoch+1, epoch_loss, epoch_acc))
        
        
        batches = data_gen(conv_feat=conv_feat_val,labels=labels_val,paths=paths_val,shuffle=shuffle)
        total = 0
        running_loss = 0.0
        running_corrects = 0
        
        for inputs,classes,paths in batches:
            inputs,classes = gpu(torch.from_numpy(inputs)),gpu(torch.from_numpy(classes))       
                                        
            fea_embeddings, ing_embeddings, cut_embeddings  = model(inputs)
            loss = 0.0
            preds = []
            p_ingres,p_cuts = pooling(ing_embeddings, cut_embeddings)
            for i in range(fea_embeddings.size(0)):
                fea_emb = fea_embeddings[i]
                ing_emb = ing_embeddings[i]
                cut_emb = cut_embeddings[i]
                p_ingre = p_ingres[i]
                p_cut = p_cuts[i]
                t_ingre = Get_ingre(paths[i])
                t_cut = Get_cut(t_ingre)
                # loss
                l = loss_function(p_ingre, p_cut, t_ingre, t_cut)
                # similarity, retrival           
                pred = pred_dish(p_ingre,p_cut,All_recipe)
#                 loss.append(l)
                loss += l
                preds.append(pred)
                
#             loss = torch.FloatTensor(loss)
#             preds = torch.FloatTensor(preds)
            preds = gpu(torch.from_numpy(np.asarray(preds)))
#             loss = gpu(torch.autograd.Variable(torch.from_numpy(np.asarray([loss]))))

            # statistics
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == classes.data)
        epoch_loss = running_loss / size_val
        epoch_acc = running_corrects.data.item() / size_val
        LOSS_V.append(epoch_loss)
        ACC_V.append(epoch_acc)
        print('Epoch: {} Validation Loss: {:.4f} Validation Acc: {:.4f}'.format(
                     epoch+1, epoch_loss, epoch_acc))
        
#     return 
    return LOSS, ACC, LOSS_V, ACC_V
#     return LOSS, ACC

In [36]:
dset_sizes['train']

66071

In [35]:
cutDim = 14
ingDim = 353
cc_rate = 0.2

LOSS, ACC, LOSS_V, ACC_V = (train_model_emb(model=gpu(embedding()),
            size_train=dset_sizes['train'],size_val=dset_sizes['val'],
            conv_feat_train=conv_feat_tr,labels_train=labels_tr,
            paths_train=paths_tr,
            conv_feat_val=conv_feat_v,labels_val=labels_v,
            paths_val=paths_v,batch_size = batch_size,
            epochs=30,optimizer=optimizer,train=True,shuffle=True))

1-th over 64 of batch 1
2-th over 64 of batch 1
3-th over 64 of batch 1
4-th over 64 of batch 1
5-th over 64 of batch 1
6-th over 64 of batch 1
7-th over 64 of batch 1
8-th over 64 of batch 1
9-th over 64 of batch 1
10-th over 64 of batch 1
11-th over 64 of batch 1
12-th over 64 of batch 1
13-th over 64 of batch 1
14-th over 64 of batch 1
15-th over 64 of batch 1
16-th over 64 of batch 1
17-th over 64 of batch 1
18-th over 64 of batch 1
19-th over 64 of batch 1
20-th over 64 of batch 1
21-th over 64 of batch 1
22-th over 64 of batch 1
23-th over 64 of batch 1
24-th over 64 of batch 1
25-th over 64 of batch 1
26-th over 64 of batch 1
27-th over 64 of batch 1
28-th over 64 of batch 1
29-th over 64 of batch 1
30-th over 64 of batch 1
31-th over 64 of batch 1
32-th over 64 of batch 1
33-th over 64 of batch 1
34-th over 64 of batch 1
35-th over 64 of batch 1
36-th over 64 of batch 1
37-th over 64 of batch 1
38-th over 64 of batch 1
39-th over 64 of batch 1
40-th over 64 of batch 1
41-th ove

KeyboardInterrupt: 

In [ ]:
plt.figure()
E = np.arange(50)+1
plt.plot(E,LOSS,label='training')
plt.plot(E,LOSS_V,label='validation')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss")
plt.legend()
plt.show()
plt.figure()
plt.plot(E,ACC,label='trianing')
plt.plot(E,ACC_V,label='validation')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training accuracy")
plt.legend()
plt.show()